In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers[torch] accelerate -U
!pip install datasets

In [3]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/트랜스포머/simple exam/fine_tune/cola_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/트랜스포머/simple exam/fine_tune/cola_test.csv')
vali = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/트랜스포머/simple exam/fine_tune/cola_validation.csv')
print(train.head())
print(test.head())

                                            sentence  label  idx
0  Our friends won't buy this analysis, let alone...      1    0
1  One more pseudo generalization and I'm giving up.      1    1
2   One more pseudo generalization or I'm giving up.      1    2
3     The more we study verbs, the crazier they get.      1    3
4          Day by day the facts are getting murkier.      1    4
                                            sentence  label  idx
0                      Bill whistled past the house.     -1    0
1              The car honked its way down the road.     -1    1
2                    Bill pushed Harry off the sofa.     -1    2
3               the kittens yawned awake and played.     -1    3
4  I demand that the more John eats, the more he ...     -1    4


In [4]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [5]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Hugging Face의 Dataset 객체로 변환
train_dataset = Dataset.from_pandas(train)
vali_dataset = Dataset.from_pandas(vali)
def preprocess_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True,max_length=512)
train_dataset = train_dataset.map(preprocess_function, batched=True)
vali_dataset = vali_dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

In [6]:
batch_size = 4
logging_steps = len(train_dataset["sentence"]) // batch_size
model_name = f"distilbert-finetuned"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=3,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  save_strategy="epoch",
                                  load_best_model_at_end=True,
                                  log_level="error")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=train_dataset,
                  eval_dataset=vali_dataset,
                  tokenizer=tokenizer)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.567100,0.520120,0.783317,0.759260
2,0.433100,0.718158,0.810163,0.802982
3,0.279900,0.936014,0.808245,0.798016


TrainOutput(global_step=6414, training_loss=0.4264784222518102, metrics={'train_runtime': 1402.7603, 'train_samples_per_second': 18.288, 'train_steps_per_second': 4.572, 'total_flos': 3398186177722368.0, 'train_loss': 0.4264784222518102, 'epoch': 3.0})

In [9]:
pre=trainer.predict(vali_dataset)
pre.metrics

{'test_loss': 0.52012038230896,
 'test_accuracy': 0.7833173537871524,
 'test_f1': 0.7592601517657882,
 'test_runtime': 20.1172,
 'test_samples_per_second': 51.846,
 'test_steps_per_second': 12.974}

In [44]:
test_dataset = Dataset.from_pandas(test)
test_dataset = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

In [45]:
ttt=trainer.predict(test_dataset)
ttt.metrics

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
